# Setup.ipynb

Written by Taiob  Ali

PASS Data Community SUMMIT 2022

SqlWorldWide.com

## Setup Demo

Setup section of the notebook will create:

- A resource group
- One logical SQL server
- Empty sample databases
- Create Log AnalyticsWorkspace
- Setting up an action group(Azure Alert)
- Creates a Log Alert Rule (Scheduled Query Rule type) using Kusto Query Language
- Clean up code at the end

<mark>Script can take between 5~8 minutes during my test. Mileage will vary in your case</mark>

Credit: [Azure PowerShell samples for Azure SQL Database and Azure SQL Managed Instance](https://learn.microsoft.com/en-us/azure/azure-sql/database/powershell-script-content-guide?view=azuresql&tabs=single-database)

<span style="color: rgb(23, 23, 23); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(226, 218, 241);">PowerShell 7 and later is the recommended version of PowerShell for use with Az PowerShell on all platforms.</span>

To check the version of PowerShell running on your machine, run the following command.

If you have an outdated version, upgrade from:  
[https://github.com/PowerShell/PowerShell](https://github.com/PowerShell/PowerShell)

In [ ]:
$PSVersionTable.PSVersion

[Install the Azure Az PowerShell module](https://learn.microsoft.com/en-us/powershell/azure/install-az-ps?view=azps-9.1.0&viewFallbackFrom=azps-5.4.0) if you do not have it.

In [ ]:
Import-Module -name Az 

Setup your [$DebugPreference](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_preference_variables?view=powershell-7.2#debugpreference)

This example shows the effect of `$DebugPreference` with the Continue value. The debug message is displayed and the command continues to process.

In [ ]:
$DebugPreference = "Continue"
Write-Debug -Message "Hello, World"

Sign in to Azure

In [ ]:
Connect-AzAccount 

If you need to see the list of your subscription

<span style="color: rgb(106, 153, 85);">$SubscriptionList=Get-AzSubscription</span><span style="color: rgb(106, 153, 85);">$SubscriptionList</span>

Use below code if you have multiple subscription and you want to use a particular one

In [ ]:
Set-AzContext -SubscriptionId '6f8db000-8416-43f0-a2db-cbfb7c945982'

Breaking change warnings are a means for the cmdlet authors to communicate with the end users any upcoming breaking changes in the cmdlet. Most of these changes will be taking effect in the next breaking change release. 

How do I get rid of the warnings? 

To suppress these warning messages, set the environment variable 'SuppressAzurePowerShellBreakingChangeWarnings' to 'true'.

In [ ]:
Set-Item Env:\SuppressAzurePowerShellBreakingChangeWarnings "true"

Declare Variables

In [ ]:
# The region and resource group name
$resourceGroupName = "sqlpassdemo"
$rgLocation = "East US"  
# Log Analytics workspace name
$workspaceName = "sqlpassws"
# The logical server name: Use a random value or replace with your own value (do not capitalize)
$sqlServerName = "sqlpassdemoservertaiob"
# The database name
$alertDemoDatabase = "sqlpassdemodatabase"

# Set an admin login and password for your database
# The login information for the server
$adminlogin = "taiob"
#Replace with password file location
$password = Get-Content "C:\password.txt"

# The ip address range that you want to allow to access your server - change as appropriate
$ipinfo = Invoke-RestMethod http://ipinfo.io/json 
$startip = $ipinfo.ip
$endip = $ipinfo.ip 

Check if Resource group exists. If exist delete the resource group.

Create a new Resource group

In [ ]:
$resGrpChk = Get-AzResourceGroup `
    -Name $resourceGroupName `
    -ev notPresent `
    -ea 0

if ($resGrpChk) {  
    #Delete resource group
    Remove-AzResourceGroup `
        -Name $resourceGroupName -Confirm   
    Write-Host 'Resource group deleted' `
        -fore white `
        -back green
}

New-AzResourceGroup `
    -Name $resourceGroupName `
    -Location "East US 2"    
Write-Host 'Resource group created' `
    -fore white `
    -back green

- Creating a Azure SQL server
- Configure server firewall rule for the server
- Create an empty database

In [ ]:
New-AzSqlServer `
    -ResourceGroupName $resourceGroupName `
    -ServerName $sqlServerName `
    -Location $rgLocation `
    -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
    -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))

New-AzSqlServerFirewallRule `
    -ResourceGroupName $resourceGroupName `
    -ServerName $sqlServerName `
    -FirewallRuleName "TaiobDesktop" `
    -StartIpAddress $startip `
    -EndIpAddress $endip

New-AzSqlServerFirewallRule `
    -ServerName $sqlServerName `
    -ResourceGroupName $resourceGroupName  `
    -AllowAllAzureIPs

New-AzSqlDatabase  `
    -ResourceGroupName $resourceGroupName `
    -ServerName $sqlServerName `
    -DatabaseName $alertDemoDatabase `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 

- Create a log analytics workspace
- Enable solutions
- Setup diagonstic for the Azure SQL Database, detination to the log analytics workspace

In [ ]:
#Create a log analytics workspace
New-AzOperationalInsightsWorkspace `
    -Location $rgLocation `
    -Name $workspaceName `
    -ResourceGroupName $resourceGroupName

# List of solutions to enable for the log analytics workspace you create above
$Solutions = "Security", "Updates", "SQLAssessment", "AzureSQLAnalytics"
# Add solutions
foreach ($solution in $Solutions) {
    Set-AzOperationalInsightsIntelligencePack `
    -ResourceGroupName $resourceGroupName `
    -WorkspaceName $workspaceName `
    -IntelligencePackName $solution -Enabled $true
}

# Setting up Azure SQL Database to send diagonstic data to log analytics workspace created above
$workspaceId = (Get-AzResource -name $workspaceName).ResourceId
$databaseId = (Get-AzsqlDatabase -ServerName $sqlServerName -DatabaseName $alertDemoDatabase -ResourceGroupName $resourceGroupName).ResourceId

Make sure your subscription is registered for microsoft.insights 

If you get an error about 'not registered' read this 

[https://kasunkodagoda.com/2019/03/10/registering-azure-resource-providers-to-enable-azure-features/](https://kasunkodagoda.com/2019/03/10/registering-azure-resource-providers-to-enable-azure-features/)

In [ ]:
Set-AzDiagnosticSetting `
    -WorkspaceId $workspaceId `
    -ResourceId $databaseId `
    -Enabled $True `
    -Name "sqlpassdemo"

Setting up action group to use with the alert
Change the value of ```
$phoneNumber
``` and ```
$emailaddress
``` before running this script

In [ ]:
$emailaddress = 'taiob@sqlworldwide.com'
$phoneNumber = 9780000000
$emailDBA = 
New-AzActionGroupReceiver `
    -Name 'emailDBA' `
    -EmailAddress $emailaddress
$smsDBA = 
New-AzActionGroupReceiver  `
    -Name 'smsDBA' -SmsReceiver `
    -CountryCode '1' `
    -PhoneNumber $phoneNumber 
$phoneDBA = 
New-AzActionGroupReceiver `
    -Name 'phoneDBA' `
    -VoiceReceiver -VoiceCountryCode '1' `
    -VoicePhoneNumber $phoneNumber 
 
Set-AzActionGroup `
    -Name 'notifydbadeadlock' `
    -ResourceGroupName $resourceGroupName `
    -ShortName 'deadlock' `
    -Receiver $emailDBA,$smsDBA,$phoneDBA

# Retreive action group id
$actionGroupId =(Get-AzResource -name 'notifydbadeadlock').ResourceId

I am using a simple Kusto query here for demo purpose.
In production I will use this one:
```
AzureMetrics
| where ResourceProvider == "MICROSOFT.SQL"
| where TimeGenerated >=ago(60min)
| where MetricName in ('deadlock')
| parse _ResourceId with * "/microsoft.sql/servers/" Resource // subtract Resource name for _ResourceId
| summarize Deadlock_max_60Mins = max(Maximum) by Resource, MetricName
```

Creating the Azure alert

In [ ]:
$source =
New-AzScheduledQueryRuleSource  `
    -Query "AzureMetrics | where MetricName == 'deadlock' "  `
    -DataSourceId $workspaceId

$schedule = 
New-AzScheduledQueryRuleSchedule `
    -FrequencyInMinutes 15 `
    -TimeWindowInMinutes 30
    
$triggerCondition = 
New-AzScheduledQueryRuleTriggerCondition `
    -ThresholdOperator  "GreaterThan" `
    -Threshold 0 
    
$aznsActionGroup = 
New-AzScheduledQueryRuleAznsActionGroup `
    -ActionGroup $actionGroupId `
    -EmailSubject "Deadlock Found" 

$alertingAction = 
New-AzScheduledQueryRuleAlertingAction `
    -AznsAction $aznsActionGroup `
    -Severity "3" `
    -Trigger $triggerCondition

New-AzScheduledQueryRule `
    -ResourceGroupName $resourceGroupName `
    -Location $rgLocation `
    -Action $alertingAction `
    -Enabled $true `
    -Description "This alert will be fired when a deadlock is found in last 30 minutes " `
    -Source $source `
    -Schedule $schedule `
    -Name "Found Deadlock Alert"

Enable the log analytics auditing policy of an Azure SQL server  

In [ ]:
Set-AzSqlServerAudit `
    -ResourceGroupName $resourceGroupName `
    -ServerName $sqlServerName `
    -LogAnalyticsTargetState Enabled `
    -WorkspaceResourceId $workspaceId

Run following scripts to continue set up 

- Setup2.sql 
- Setup3.sql

Clean up (at the end fo the session) by removing resource group name

In [ ]:
Remove-AzResourceGroup -ResourceGroupName $resourceGroupName -Force